In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm as tqdm

# 1

In [3]:
players = pd.read_pickle('chgk/players.pkl')
tournaments = pd.read_pickle('chgk/tournaments.pkl')
results = pd.read_pickle('chgk/results.pkl')

In [4]:
data_tournaments = pd.DataFrame(tournaments.values())

In [6]:
train_tourn = []
test_tourn = []

for value in tournaments.values():
    current_year = pd.to_datetime(value['dateStart']).year
    
    if current_year == 2019:
        train_tourn.append(value)
    if current_year == 2020:
        test_tourn.append(value)

In [7]:
len(train_tourn), len(test_tourn) # После фильтрации по годам 

(687, 418)

In [8]:
def check_None_data(train_tourn):
    train = []
    
    for value in train_tourn:
        tournament_id = value['id']
        info = results[tournament_id]
        if info:
            count_num = 0
            for num in info:
                mask = num.get('mask', None)
                team = num.get('teamMembers', None)
                if (mask is not None) and (team is not None):
                    count_num += 1

            if count_num == len(info):
                train.append(value)
    
    return train

In [9]:
train = check_None_data(train_tourn)
test = check_None_data(test_tourn)

In [10]:
len(train), len(test) # После филтрации по пустым маскам и членов команды

(671, 169)

# 2

Получим завивсимость игрок-вопрос-ответ

In [11]:
def get_train_data(tournaments):
    ids, question_ids, answer = [], [], []
    current_id = 1
    for value in tqdm(tournaments):
        tournament_id = value['id']
        info = results[tournament_id]
        mask_size = sum(value['questionQty'].values())
        ques_id_range = np.arange(current_id, current_id+mask_size)
        for team in info:
            mask = team['mask']
            team_players_id = [i['player']['id'] for i in team['teamMembers']]
            if '?' not in mask and len(mask) == mask_size:
                mask = mask.replace('X', '')
                mask_items = np.array(list(map(int, list(mask))))
                for q_ind, item in enumerate(mask_items):
                    for player in team_players_id:
                        ids.append(player)
                        question_ids.append(ques_id_range[q_ind])
                        answer.append(item)
        current_id += mask_size
                    
    return ids, question_ids, answer

In [12]:
ids, question_ids, answer = get_train_data(train)

In [13]:
df_train = pd.DataFrame({'player_id': ids, 'question_id': question_ids, 'answer': answer})
df_train.head()

,player_id,question_id,answer
0,6212,1,1
1,18332,1,1
2,18036,1,1
3,22799,1,1
4,15456,1,1


In [17]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
import scipy.sparse as scp

In [28]:
encoder_pla = OneHotEncoder()
encoder_que = OneHotEncoder()

player_data = encoder_pla.fit_transform(df_train[['player_id']])
question_data = encoder_que.fit_transform(df_train[['question_id']])
train = scp.hstack([player_data, question_data])

In [29]:
player_data.shape, question_data.shape, train.shape

((17685248, 57329), (17685248, 32905), (17685248, 90234))

In [30]:
target = df_train.answer.to_numpy()

In [31]:
logreg = LogisticRegression()
logreg.fit(train, target)

/Users/pavel/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [53]:
players_rank = pd.DataFrame({'id': encoder_pla.categories_[0], 'rank': logreg.coef_[0][:len(encoder_pla.categories_[0])]})

In [56]:
players_rank = players_rank.merge(pd.DataFrame(pd.DataFrame(players.values())), how='left', 
                                                                                left_on='id',
                                                                                right_on='id')

In [70]:
players_rank.sort_values('rank', ascending=False)[:20]

,id,rank,name,patronymic,surname
3859,27403,3.941128,Максим,Михайлович,Руссо
604,4270,3.845133,Александра,Владимировна,Брутер
3931,27822,3.484024,Михаил,Владимирович,Савченков
4048,28751,3.423060,Иван,Николаевич,Семушин
5176,37047,3.345671,Мария,Алексеевна,Юнгер
4236,30152,3.308988,Артём,Сергеевич,Сорожкин
2926,20691,3.292816,Станислав,Григорьевич,Мереминский
6715,56647,3.275960,Наталья,Евгеньевна,Горелова
4790,34328,3.249853,Михаил,Сергеевич,Царёв
2567,18036,3.243332,Михаил,Ильич,Левандовский


# 3

In [61]:
from scipy.stats import spearmanr, kendalltau

предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд.
-Будем располагать команды благодаря усредненному рейтингу всех ее игроков.

In [ ]:
def correlation_calc(test):
    result_spear, result_kendal = [], []

    for value in tqdm(test):
        id_team, position = [], []
        rank_local = []
        tournament_id = value['id']
        info = results[tournament_id]
        if len(info) > 1:
            for num in info:
                id_team.append(num['team']['id'])
                position.append(num['position'])

                mean_rank_team = []
                for members in num['teamMembers']:
                    if len(players_rank[players_rank['id'] == members['player']['id']]) != 0:
                        mean_rank_team.append(players_rank[players_rank['id'] == members['player']['id']]['rank'].values[0])
                    else:
                        mean_rank_team.append(0)

                rank_local.append(np.mean(mean_rank_team))

            current_tournament = pd.DataFrame({'id_team': id_team, 'position': position, 'predict_rank': rank_local}).sort_values('predict_rank', ascending=False)
            result_spear.append(spearmanr(current_tournament['position'], position)[0])
            result_kendal.append(kendalltau(current_tournament['position'], position)[0])
    
    return np.mean(result_spear), np.mean(result_kendal)

In [131]:
spear, kend = correlation_calc(test)

In [132]:
print(f'Корреляция Спирмена:{spear}')
print(f'Корреляция Кендела:{kend}')

Корреляция Спирмена:0.7316470783348838
Корреляция Кендела:0.5864375064336904
